In [1]:
import itertools
import findspark
findspark.init()
from pyspark import SparkContext, SparkConf
import sys


In [2]:
def get_original_itemset_counts(basket):
    if set_size > 1:
        sets = itertools.combinations(basket[1], set_size)

    else:
        sets = basket[1]

    return list(map(lambda x: (x, 1), sets))




In [3]:
def get_partition_itemset_candidates(bucket):
    items = {}
    for basket in bucket:
        if len(basket[1]) < set_size:
            continue

        if set_size > 1:
            sets = itertools.combinations(basket[1], set_size)

        else:
            sets = basket[1]

        for item in sets:
            if item not in items:
                items[item] = 0

            items[item] += 1

    print(items)
    for item in items:
        if items[item] >= partial_support:
            yield item


In [4]:
def get_candidates_list(candidate_baskets):
    candidate_sets = candidate_baskets.mapPartitions(get_partition_itemset_candidates)
    candidate_sets_list = candidate_sets.distinct().collect()
    return candidate_sets_list


In [5]:
def get_original_frequent_sets(original_baskets, candidate_list):
    original_freq_sets = original_baskets \
        .flatMap(get_original_itemset_counts) \
        .filter(lambda x: x[0] in candidate_list) \
        .reduceByKey(lambda a, b: a + b) \
        .filter(lambda x: x[1] >= support)\
        .map(lambda x: (x[0]))\
        .collect()

    return original_freq_sets


In [10]:
def write_results(result_candidates, result_frequent_itemsets):
    with open('jupyter_test_results', 'w') as results_file:
        results_file.write('Candidates:\n')
        output = ''
        for single_cad in sorted(result_candidates[0]):
            output += '(' + str(single_cad) + '),'

        results_file.write(output[-1])

        for cand_set in result_candidates[1:]:
            results_file.write(str(sorted(cand_set))[1:-1] + '\n\n')

        results_file.write('Frequent Itemsets:\n')
        output = ''
        for single_item in sorted(result_frequent_itemsets[0]):
            output += '(' + str(single_item) + '),'

        results_file.write(output[-1])

        for freq_set in result_frequent_itemsets[1:]:

            results_file.write(str(sorted(freq_set))[1:-1] + '\n\n')


In [7]:
case = '1'
conf = SparkConf()
conf.set("spark.driver.memory", "4g")
conf.set("spark.executor.memory", "4g")
conf.setMaster('local[8]')
conf.setAppName('Assignment_2')
sc = SparkContext.getOrCreate(conf)

data_path = 'asnlib/publicdata/'

data = sc.textFile(data_path + 'small1.csv').map(lambda x: x.split(',')).map(lambda x: (x[0], x[1]))
header = data.first()
raw_data = data.filter(lambda x: x != header)


if case == '2':
    raw_data = raw_data.map(lambda x: (x[1], x[0]))

baskets = raw_data.distinct().groupByKey().map(lambda x: (x[0], sorted(list(x[1]))))
print(baskets.collect())


[('1', ['100', '101', '102', '98']), ('4', ['101', '102', '103', '97', '99']), ('8', ['102', '103', '104', '97', '98', '99']), ('9', ['97', '98', '99']), ('10', ['97', '98']), ('12', ['100', '101', '102', '105', '106', '107', '108', '98']), ('16', ['100', '101', '98', '99']), ('19', ['102', '97', '98']), ('14', ['97']), ('17', ['97', '99']), ('2', ['100', '101', '97', '99']), ('3', ['102', '103', '105', '97', '98', '99']), ('5', ['97', '98']), ('6', ['101', '102']), ('7', ['101', '97', '99']), ('11', ['97', '98', '99']), ('13', ['100', '101', '102', '103', '105', '106', '107', '108', '98', '99']), ('15', ['101', '97', '99']), ('18', ['97', '98', '99'])]


In [8]:
support = 4
partial_support = support // baskets.getNumPartitions()

In [16]:
candidates = []
frequent_itemsets = []
set_size = 1
while True:
    current_candidates = get_candidates_list(candidate_baskets=baskets)
    # current_candidates = []
    current_frequent_itemsets = get_original_frequent_sets(
        original_baskets=baskets,
        candidate_list=current_candidates
    )
    if not current_candidates:
        break
    candidates.append(current_candidates)
    if current_frequent_itemsets:
        frequent_itemsets.append(current_frequent_itemsets)
    set_size += 1

In [19]:
def write_results(result_candidates, result_frequent_itemsets):
    with open('jupyter_test_results', 'w') as results_file:
        results_file.write('Candidates:\n')
        output = ''
        for single_cad in sorted(result_candidates[0]):
            output += '(' + str(single_cad) + '),'

        results_file.write(output[-1])

        for cand_set in result_candidates[1:]:
            results_file.write(str(sorted(cand_set))[1:-1] + '\n\n')

        results_file.write('Frequent Itemsets:\n')
        output = ''
        for single_item in sorted(result_frequent_itemsets[0]):
            output += '(' + str(single_item) + '),'

        results_file.write(output[-1])

        for freq_set in result_frequent_itemsets[1:]:

            results_file.write(str(sorted(freq_set))[1:-1] + '\n\n')

write_results(result_candidates=candidates, result_frequent_itemsets=frequent_itemsets)

In [22]:
text = ''
for single_cad in sorted(candidates[0]):
    text += '(\'' + str(single_cad) + '\'),'

In [23]:
text

"('100'),('101'),('102'),('103'),('105'),('97'),('98'),('99'),"

In [ ]:
text[:-1]

In [17]:
candidates

[['100', '102', '98', '101', '103', '97', '99', '105'],
 [('100', '102'),
  ('100', '98'),
  ('102', '98'),
  ('101', '99'),
  ('103', '97'),
  ('103', '99'),
  ('97', '99'),
  ('101', '97'),
  ('103', '105'),
  ('105', '99'),
  ('100', '101'),
  ('101', '102'),
  ('101', '98'),
  ('102', '103'),
  ('102', '97'),
  ('102', '99'),
  ('97', '98'),
  ('98', '99'),
  ('100', '99'),
  ('102', '105'),
  ('103', '98'),
  ('105', '98')],
 [('100', '101', '102'),
  ('100', '101', '98'),
  ('101', '102', '98'),
  ('103', '97', '99'),
  ('102', '97', '98'),
  ('101', '97', '99'),
  ('102', '103', '98'),
  ('102', '105', '98'),
  ('102', '98', '99'),
  ('103', '105', '99'),
  ('100', '102', '98'),
  ('102', '103', '97'),
  ('102', '103', '99'),
  ('102', '97', '99'),
  ('97', '98', '99'),
  ('100', '101', '99'),
  ('102', '103', '105'),
  ('102', '105', '99'),
  ('103', '105', '98'),
  ('103', '98', '99'),
  ('105', '98', '99')],
 [('102', '103', '105', '98'),
  ('102', '103', '98', '99'),
  ('102

In [18]:
frequent_itemsets

[['100', '102', '98', '101', '103', '97', '99'],
 [('100', '98'),
  ('102', '98'),
  ('101', '97'),
  ('101', '99'),
  ('103', '99'),
  ('97', '99'),
  ('100', '101'),
  ('101', '102'),
  ('101', '98'),
  ('102', '103'),
  ('102', '97'),
  ('102', '99'),
  ('97', '98'),
  ('98', '99')],
 [('100', '101', '98'),
  ('101', '97', '99'),
  ('102', '103', '99'),
  ('97', '98', '99')]]